In [1]:
import os.path as op

import numpy as np
import mne
import mne.datasets.somato as somato
from mne_bids import write_raw_bids, make_bids_basename
from mne_bids.utils import print_dir_tree

In [2]:
# Path to the raw data of somato dataset
fif_path = op.join(somato.data_path(), 'MEG', 'somato')

In [3]:
# Load the raw data file
raw = mne.io.read_raw_fif(op.join(fif_path, 'sef_raw_sss.fif'))

Opening raw data file /home/stefanappelhoff/mne_data/MNE-somato-data/MEG/somato/sef_raw_sss.fif...
    Range : 237600 ... 506999 =    791.189 ...  1688.266 secs
Ready.
Current compensation grade : 0


In [4]:
# Get the events, and assign an ID
events = mne.find_events(raw)
event_id = dict()
for val in np.unique(events[:, -1]):
    event_id['somato_event{}'.format(val)] = val
    
event_id

111 events found
Event IDs: [1]


{'somato_event1': 1}

In [5]:
# Make a bids name for the raw file
bids_basename = make_bids_basename(subject='01', task='somato')

In [6]:
# Make a location for the BIDS formatted data
somato_parent, somato_name = op.split(somato.data_path())
output_path = op.join(somato_parent, 'bids_' + somato_name)

In [7]:
# Write the data
write_raw_bids(raw, bids_basename, output_path, events, event_id, overwrite=True)

Opening raw data file /home/stefanappelhoff/mne_data/MNE-somato-data/MEG/somato/sef_raw_sss.fif...
    Range : 237600 ... 506999 =    791.189 ...  1688.266 secs
Ready.
Current compensation grade : 0
Creating folder: /home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/meg

Writing '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/participants.tsv'...

participant_id	age	sex
sub-01	n/a	n/a

Writing '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/participants.json'...

{
    "participant_id": {
        "Description": "Unique participant identifier"
    },
    "age": {
        "Description": "Age of the participant at time of testing",
        "Units": "years"
    },
    "sex": {
        "Description": "Biological sex of the participant",
        "Levels": {
            "F": "female",
            "M": "male"
        }
    }
}

Writing '/home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/sub-01_scans.tsv'...

filename	acq_time
meg/sub-01_task-somato_meg.fif	2007-07-05T

/home/stefanappelhoff/miniconda3/envs/mne/lib/python3.6/site-packages/mne_bids-0.2.dev0-py3.6.egg/mne_bids/write.py:418: UserWarning: No line frequency found, defaulting to 50 Hz
  warn('No line frequency found, defaulting to 50 Hz')
/home/stefanappelhoff/miniconda3/envs/mne/lib/python3.6/site-packages/mne_bids-0.2.dev0-py3.6.egg/mne_bids/write.py:921: RuntimeWarning: This filename (/home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/meg/sub-01_task-somato_meg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw.save(bids_fname, overwrite=True, split_naming='neuromag')


Closing /home/stefanappelhoff/mne_data/bids_MNE-somato-data/sub-01/meg/sub-01_task-somato_meg.fif [done]


'/home/stefanappelhoff/mne_data/bids_MNE-somato-data'

In [8]:
print_dir_tree(output_path)

|bids_MNE-somato-data
|--- participants.tsv
|--- dataset_description.json
|--- participants.json
|--- sub-01
|------ sub-01_scans.tsv
|------ meg
|--------- sub-01_task-somato_channels.tsv
|--------- sub-01_task-somato_meg.json
|--------- sub-01_task-somato_events.tsv
|--------- sub-01_task-somato_meg.fif
|--------- sub-01_coordsystem.json
